# Normal implementation

In [1]:
from __future__ import annotations

import os
import re
import json
import yaml
import csv
import math
import time
import logging
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed

# LangChain (modern imports)
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
import backoff
import numpy as np
from langchain_community.embeddings import SentenceTransformerEmbeddings

# ========== Logging ==========
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s"
)

# ========== IO helpers ==========

def load_yaml_job_description(path: str) -> Dict[str, Any]:
    with open(path, 'r', encoding='utf-8') as f:
        jd = yaml.safe_load(f) or {}
    return jd


import logging

def _read_pdf_pdfminer_md(file_path: str) -> str:
    """
    Extract PDF text as Markdown using pdfminer.six.
    """
    try:
        from pdfminer.high_level import extract_text
        text = extract_text(file_path) or ""
        if text.strip():
            # Convert to markdown-friendly (basic)
            return "\n".join([line.strip() for line in text.splitlines() if line.strip()])
        return ""
    except Exception as e:  # pragma: no cover
        logging.warning(f"pdfminer failed on {file_path}: {e}")
        return ""


def _read_pdf_pypdf_md(file_path: str) -> str:
    """
    Extract PDF text as Markdown fallback using pypdf.
    """
    try:
        from pypdf import PdfReader
    except Exception:  # pragma: no cover
        return ""

    try:
        text = []
        with open(file_path, "rb") as f:
            reader = PdfReader(f)
            for page in reader.pages:
                try:
                    page_text = page.extract_text() or ""
                    if page_text.strip():
                        # Markdown-friendly formatting
                        text.append("\n".join([ln.strip() for ln in page_text.splitlines() if ln.strip()]))
                except Exception:
                    text.append("")
        return "\n".join(text)
    except Exception as e:
        logging.warning(f"pypdf failed on {file_path}: {e}")
        return ""


def load_resume_markdown(file_path: str) -> str:
    """
    Load resume and return Markdown text.
    Priority: pdfminer -> pypdf -> warn if fails.
    """
    file_path = str(file_path)
    if file_path.lower().endswith('.txt'):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        except Exception as e:
            logging.warning(f"Could not read TXT {file_path}: {e}")
            return ""
    elif file_path.lower().endswith('.pdf'):
        text = _read_pdf_pdfminer_md(file_path)
        if not text.strip():
            text = _read_pdf_pypdf_md(file_path)
        if not text.strip():
            logging.warning(f"Could not extract text from PDF {file_path}")
        return text
    else:
        logging.warning(f"Unsupported resume format: {file_path}")
        return ""



def safe_candidate_name_from_file(file_name: str) -> str:
    stem = Path(file_name).stem
    return re.sub(r"[^A-Za-z0-9_.-]", "_", stem)


# ========== Pydantic schema for structured output ==========

class MatchReport(BaseModel):
    matched_required_skills: List[str] = []
    missing_required_skills: List[str] = []
    matched_optional_skills: List[str] = []
    education_match: str
    experience_match: str
    keywords_matched: List[str] = []
    soft_skills_match: List[str] = []
    resume_summary: str
    match_score: float = Field(ge=0, le=1)
    city_tier_match: bool
    longest_tenure_months: int
    final_score: int = Field(ge=0, le=100)
    # --- extended, optional fields for evaluation criteria ---
    detected_city: Optional[str] = None
    detected_city_tier: Optional[int] = None  # 1/2/3 if the model can infer
    max_job_gap_months: Optional[int] = None
    stability_score: Optional[float] = Field(default=None, ge=0, le=1)

# ========== Prompt ==========

# PROMPT = ChatPromptTemplate.from_messages([
#     (
#         "system",
#         "You are an expert technical recruiter and data scientist. "
#         "Your job is to read a job description (JD) and a resume chunk, then return a STRICT JSON object matching the schema. "
#         "Be precise, consistent, and terse. If the information is not present, return a sensible null/empty value rather than guessing. "
#         "NEVER add commentary, markdown, or keys not in the schema."
#     ),
#     (
#         "user",
#         "<OBJECTIVE>\n"
#         "Evaluate the resume against the JD and produce high-quality, schema-valid JSON capturing skills, education, experience fit, city-tier & gaps, longest tenure, and a calibrated final_score.\n\n"
#         "<INPUTS>\n"
#         "Job Description (YAML):\n{job_description}\n\n"
#         "Resume chunk:\n{resume_text}\n\n"
#         "<SCHEMA AND CONSTRAINTS>\n"
#         "You must return a single JSON object with the following keys and constraints:\n"
#         "- matched_required_skills: string[] (subset of JD.required_skills that appear in the resume; normalize case and aliases like js->javascript, py->python, torch->pytorch)\n"
#         "- missing_required_skills: string[] (skills from JD.required_skills not evidenced in the resume chunk)\n"
#         "- matched_optional_skills: string[] (subset of JD.optional_skills found)\n"
#         "- education_match: string (short justification or 'false' if not met; keep to <= 1 sentence)\n"
#         "- experience_match: string (short justification or 'false' if not met; <= 1 sentence)\n"
#         "- keywords_matched: string[] (notable JD keywords present in resume text)\n"
#         "- soft_skills_match: string[] (soft skills evidenced in text; e.g., communication, leadership)\n"
#         "- resume_summary: string (1–2 sentences summarizing the candidate relevant to the JD)\n"
#         "- match_score: number in [0,1] (your calibrated similarity for THIS CHUNK only)\n"
#         "- city_tier_match: boolean (true if city_tier meets or exceeds JD requirement if any; else false)\n"
#         "- longest_tenure_months: integer >= 0 (best estimate from dates in this chunk)\n"
#         "- final_score: integer in [0,100] (overall score for the FULL candidate, using rubric below; be conservative if context is incomplete)\n"
#         "- detected_city: string|null\n"
#         "- detected_city_tier: 1|2|3|null\n"
#         "- max_job_gap_months: integer|null\n"
#         "- stability_score: number in [0,1]|null (optional stability proxy)\n\n"
#         "<EVIDENCE RULES>\n"
#         "- Treat the resume chunk as ground truth. Do not infer unstated skills.\n"
#         "- Consider common aliases: js↔javascript, ts↔typescript, py↔python, torch↔pytorch, tf↔tensorflow, np↔numpy, sk↔scikit-learn. Normalize to canonical names.\n"
#         "- Date parsing: recognize ranges like 'Jan 2019 - Mar 2022', '2018–2021', '2020 to Present'. Compute tenure in months (approx). Present/current = current month. If ambiguous, be conservative.\n"
#         "- City & tier: if JD provides a city_tier map, use it; otherwise infer only if city is explicit. If unknown, set both fields null.\n\n"
#         "<RUBRIC FOR final_score (100-point scale)>\n"
#         "Use JD-provided weights if present (JD.weights). Otherwise, default weights:\n"
#         "- required skills coverage: 40%\n"
#         "- optional skills coverage: 15%\n"
#         "- experience fit (years/recency/scope): 15%\n"
#         "- education fit: 10%\n"
#         "- location fit: 5% (true if city_tier meets JD or is unspecified)\n"
#         "- stability: 10% (longest_tenure_months; full credit at 48 months; scale proportionally)\n"
#         "- diversity by city tier: 5% bonus (Tier-3 > Tier-2 > Tier-1; score 100 for T3, 60 for T2, 0 for T1)\n"
#         "Apply a mild penalty for large job gaps via the chunk-level estimate: reduce the total by ~0% (<=3m), 10% (<=6m), 25% (<=12m), 50% (>12m).\n"
#         "If JD specifies custom weights/thresholds, follow them exactly.\n\n"
#         "<ROBUSTNESS & STYLE>\n"
#         "- Keep outputs concise; arrays deduplicated and normalized to lowercase where appropriate.\n"
#         "- If data is missing in this chunk, leave fields empty/null rather than hallucinating.\n"
#         "- Never include markdown or commentary—only the JSON object.\n\n"
#         "<OUTPUT> Return ONLY the JSON object."
#     )
# ])

PROMPT = ChatPromptTemplate.from_messages([
    (
    "system",
    "You are an expert technical recruiter and data scientist. "
    "Your job is to read a job description (JD) and a resume , then return a STRICT JSON object matching the schema. "
    "Be precise, consistent, and terse. If the information is not present, return a sensible null/empty value rather than guessing. "
    "NEVER add commentary, markdown, or keys not in the schema."
),
(
    "user",
    "<OBJECTIVE>\n"
    "Evaluate the resume against the JD and produce high-quality, schema-valid JSON capturing skills, education, experience fit, city-tier & gaps, longest tenure, and a calibrated final_score.\n"
    "\n"
    "<INPUTS>\n"
    "Job Description (YAML): {job_description}\n"
    "Resume : {resume_text}\n"
    "\n"
     "<RUBRIC FOR final_score (100-point scale)>\n"
    "Weightage:\n"
    "- required skills coverage: 40%\n"
    "- optional skills coverage: 15%\n"
    "- experience fit (years/recency/scope): 15%\n"
    "- education fit: 10%\n"
    "- location fit: 5% (true if city_tier meets JD or is unspecified)\n"
    "- stability: 10% (longest_tenure_months; full credit at 48 months; scale proportionally)\n"
    "- diversity by city tier: 5% bonus (Tier-3 > Tier-2 > Tier-1; score 100 for T3, 60 for T2, 0 for T1)\n"

    "\n"
    "<SCHEMA AND CONSTRAINTS>\n"
    "You must return a single JSON object with the following keys and constraints: Strictly Don't add any extra key value pair:\n"
    "- matched_required_skills: string[] (subset of JD.required_skills that appear in the resume; normalize case and aliases like js->javascript, py->python, torch->pytorch)\n"
    "- missing_required_skills: string[] (skills from JD.required_skills not evidenced in the resume chunk)\n"
    "- matched_optional_skills: string[] (subset of JD.optional_skills found)\n"
    "- education_match: string (short justification or 'false' if not met; keep to <= 1 sentence)\n"
    "- experience_match: string (short justification or 'false' if not met; <= 1 sentence)\n"
    "- keywords_matched: string[] (notable JD keywords present in resume text)\n"
    "- soft_skills_match: string[] (soft skills evidenced in text; e.g., communication, leadership)\n"
    "- resume_summary: string (1–2 sentences summarizing the candidate relevant to the JD)\n"
    "- match_score: number in [0,1] (your calibrated similarity for THIS resume)\n"
    "- city_tier_match: boolean (true if city_tier meets or exceeds JD requirement if any; else false)\n"
    "- longest_tenure_months: integer >= 0 (The longest duration (in months) the candidate was employed in a **single company** based on their work history)\n"
    "- final_score: integer in [0,100] (An integer between 0 and 100 summarizing the overall resume match quality against the job description based on all criteria above; be conservative if context is incomplete)\n"
    "- detected_city: string|null\n"
    "- detected_city_tier: 1|2|3|null\n"
    "- city_tier_match: boolean (true if detected_city_tier meets the Job description requirement any; else false)\n"
    "- max_job_gap_months: integer|null (largest gap in months between consecutive jobs for this resume; compute as difference between next job start and previous job end)"
    "- max_job_gap_months_check: string|null (short justification if the candidate’s maximum job gap in months is <= Maximum_Job_Gap_Months specified in the Job Description; else null)"
    "\n"
    "<EVIDENCE RULES>\n"
    "- Consider common aliases: js↔javascript, ts↔typescript, py↔python, torch↔pytorch, tf↔tensorflow, np↔numpy, sk↔scikit-learn. Normalize to canonical names.\n"
    "- Date parsing: recognize ranges like 'Jan 2019 - Mar 2022', '2018–2021', '2020 to Present'. Compute tenure in months (approx). Present/current = current month. If ambiguous, be conservative.\n"
    "\n"
   
    "<ROBUSTNESS & STYLE>\n"
    "- Keep outputs concise; arrays deduplicated and normalized to lowercase where appropriate.\n"
    "- Never include markdown or commentary—only the JSON object.\n"
    "\n"
    "<OUTPUT> Return a **single valid JSON object** using this structure(without extra comments or explanations)"
)
])

# ========== Embeddings (pre-filter) ==========

def _cosine(a: np.ndarray, b: np.ndarray) -> float:
    if a is None or b is None:
        return -1.0
    na = np.linalg.norm(a); nb = np.linalg.norm(b)
    if na == 0 or nb == 0:
        return -1.0
    return float(np.dot(a, b) / (na * nb))

_embedder: Optional[SentenceTransformerEmbeddings] = None

def get_embedder() -> SentenceTransformerEmbeddings:
    global _embedder
    if _embedder is None:
        _embedder = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    return _embedder

def embed_text(text: str) -> np.ndarray:
    emb = get_embedder().embed_query(text or "")
    return np.array(emb, dtype=np.float32)


def prefilter_resumes(jd: Dict[str, Any], resume_paths: List[Path], texts: List[str], topk: Optional[int] = None, topk_frac: float = 0.4) -> List[Tuple[Path, float]]:
    """Rank resumes by embedding similarity to the JD and return the top subset.
    If topk is None, select ceil(len(resumes) * topk_frac). Never fewer than 1.
    """
    jd_text = yaml.dump(jd, sort_keys=False)
    jd_vec = embed_text(jd_text)

    sims: List[Tuple[int, float]] = []
    for i, t in enumerate(texts):
        try:
            v = embed_text(t)
            sims.append((i, _cosine(jd_vec, v)))
        except Exception:
            sims.append((i, -1.0))
    sims.sort(key=lambda x: x[1], reverse=True)

    n = len(resume_paths)
    k = int(topk) if topk is not None else int(np.ceil(max(1, n) * float(topk_frac)))
    k = max(1, min(n, k))

    selected = [(resume_paths[i], score) for i, score in sims[:k]]
    return selected

# ========== LLM client ==========
key = os.getenv("OPENROUTER_API_KEY", "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")
base = os.getenv("OPENROUTER_BASE", "https://openrouter.ai/api/v1")
# model = os.getenv("MODEL_NAME", "nvidia/nemotron-nano-9b-v2:free")
model_name = "qwen/qwen2.5-vl-72b-instruct:free"

def make_llm(model: str = "gpt-4o-mini", temperature: float = 0.6):
    # print(model_name)
    llm = ChatOpenAI(model=model_name, temperature=temperature, api_key=key, base_url=base)
    # return llm.with_structured_output(MatchReport)
    return llm

# ========== Retry wrapper ==========

@backoff.on_exception(backoff.expo, Exception, max_time=90)
def call_llm_structured(structured_llm, jd_dict: Dict[str, Any], resume_text: str) -> MatchReport:
    msg = PROMPT.format(job_description=yaml.dump(jd_dict, sort_keys=False),
                        resume_text=resume_text)
    
    # IMPORTANT: for structured outputs, invoke with messages
    return structured_llm.invoke(msg)
# ========== Per-resume processing ==========

import re
import json

def clean_text_v2(text: str) -> str:
    """
    Extract the first valid JSON object/array from arbitrary text.

    - Handles Markdown fences like ```json ... ``` or plain ``` ... ```.
    - Ignores any prose before/after the JSON.
    - Returns the JSON substring (not a Python dict). If nothing parses,
      returns a best-effort substring starting at the first '{' or '['.
    """
    if not text:
        return ""

    s = text.strip().replace("\u00A0", " ")  # normalize non-breaking spaces

    # 1) Collect candidates from fenced code blocks (prefer these first).
    fence_re = re.compile(r"```(?:\w+)?\s*([\s\S]*?)\s*```", re.IGNORECASE)
    candidates = [m.group(1).strip() for m in fence_re.finditer(s)]

    # 2) Also consider the full text in case JSON isn't fenced.
    candidates.append(s)

    def balanced_json_substrings(src: str):
        """Yield substrings that are balanced JSON blocks starting at '{' or '['."""
        out = []
        i, n = 0, len(src)
        while i < n:
            ch = src[i]
            if ch in "{[":
                start = i
                stack = [ch]
                i += 1
                in_str = False
                esc = False
                while i < n:
                    c = src[i]
                    if in_str:
                        if esc:
                            esc = False
                        elif c == "\\":
                            esc = True
                        elif c == '"':
                            in_str = False
                        i += 1
                        continue
                    else:
                        if c == '"':
                            in_str = True
                        elif c in "{[":
                            stack.append(c)
                        elif c in "}]":
                            if not stack:
                                break
                            opening = stack.pop()
                            if (opening == "{" and c != "}") or (opening == "[" and c != "]"):
                                break
                            if not stack:
                                # Found a balanced block
                                out.append(src[start:i+1].strip())
                                break
                        i += 1
                # Move forward to search for the next block
                i = start + 1
            else:
                i += 1
        return out

    # 3) Try to find a substring that actually parses as JSON.
    for cand in candidates:
        for sub in balanced_json_substrings(cand):
            try:
                json.loads(sub)
                return sub
            except Exception:
                continue

    # 4) Fallback: strip fences and return from the first '{' or '[' onward.
    def _strip_fences(m):  # keep inner content
        return (m.group(1) or "").strip()

    unfenced = fence_re.sub(_strip_fences, s).strip()
    m = re.search(r"[\{\[]", unfenced)
    return unfenced[m.start():].strip() if m else ""


def process_one_resume(jd: Dict[str, Any], resume_path: Path, structured_llm) -> Optional[Dict[str, Any]]:
    text = load_resume_markdown(str(resume_path))
    if not text.strip():
        logging.warning(f"Empty/unsupported resume: {resume_path.name}; skipping.")
        return None

    try:
        r = call_llm_structured(structured_llm, jd, text)
        # print(r.content)
        cleaned_result = clean_text_v2(r.content)
        # print(cleaned_result)
        parsed = json.loads(cleaned_result)
        # print(parsed)
    except Exception as e:
        logging.error(f"LLM error on {resume_path.name}: {e}")
        return None

    candidate = safe_candidate_name_from_file(resume_path.name)
    report = {
        "candidate_name": candidate,
        "job_title": jd.get("Job_Title") or jd.get("job_title"),
        **parsed,
    }
    return report

# ========== Batch processing ==========

def process_all(job_description_file: str, resumes_folder: str, workers: int = 4, model: str = "gpt-4o-mini", topk: Optional[int] = None, topk_frac: float = 0.4) -> None:
    reports_dir = Path("reports"); reports_dir.mkdir(exist_ok=True)

    jd = load_yaml_job_description(job_description_file)
    structured_llm = make_llm(model=model)

    resume_files = [Path(resumes_folder) / fn for fn in os.listdir(resumes_folder)
                    if fn.lower().endswith((".pdf", ".txt"))]

    reports: List[Dict[str, Any]] = []

    # --- Load texts once for embedding + later scoring ---
    resume_texts = [load_resume_markdown(str(p)) for p in resume_files]

    # --- Pre-filter via embeddings ---
    ranked = prefilter_resumes(jd, resume_files, resume_texts, topk=topk, topk_frac=topk_frac)
    selected_files = [p for p, _ in ranked]
    logging.info(f"Pre-filter selected {len(selected_files)}/{len(resume_files)} resumes via embeddings")

    # Concurrency
    with ThreadPoolExecutor(max_workers=max(1, int(workers))) as ex:
        futs = {ex.submit(process_one_resume, jd, p, structured_llm): p for p in selected_files}
        for fut in as_completed(futs):
            p = futs[fut]
            try:
                rep = fut.result()
                if rep:
                    reports.append(rep)
                    # write per-candidate JSON immediately
                    out_path = Path("reports") / f"{rep['candidate_name']}_report.json"
                    with open(out_path, 'w', encoding='utf-8') as f:
                        json.dump(rep, f, indent=2, ensure_ascii=False)
            except Exception as e:
                logging.error(f"Failed {p.name}: {e}")




In [2]:
# job_description_file = r"C:\Users\Lenovo\resume_matcher\jd.yaml"
# resumes_folder = r"C:\Users\Lenovo\resume_matcher\resumes"
# topk = 2
# Path("reports").mkdir(exist_ok=True)

# t0 = time.time()
# reports = process_all(job_description_file=job_description_file, resumes_folder=resumes_folder, topk=topk)
# logging.info(f"Done processing resumes in {time.time() - t0:.1f}s")

# Langgraph agent

In [3]:
# === NEW / UPDATED SECTIONS BELOW ===
# Add these imports
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# ---------- Graph State ----------
class ResumeState(TypedDict, total=False):
    jd: Dict[str, Any]
    resume_path: str
    resume_text: str
    llm: Any
    raw_llm_output: str
    parsed_json: Dict[str, Any]
    report: Dict[str, Any]
    errors: List[str]

# ---------- Graph Nodes ----------
def node_load_resume(state: ResumeState) -> ResumeState:
    try:
        text = load_resume_markdown(state["resume_path"])
        if not text.strip():
            raise ValueError("Empty text extracted from resume")
        state["resume_text"] = text
    except Exception as e:
        errs = state.get("errors", [])
        errs.append(f"load_resume: {e}")
        state["errors"] = errs
        # You could choose to raise to stop the graph early
        raise
    return state

# def node_call_llm(state: ResumeState) -> ResumeState:
#     try:
#         r = call_llm_structured(state["llm"], state["jd"], state["resume_text"])
#         print(r.content)
#         state["raw_llm_output"] = r.content
#         cleaned = clean_text_v2(r.content)
#         state["parsed_json"] = json.loads(cleaned)
#     except Exception as e:
#         errs = state.get("errors", [])
#         errs.append(f"call_llm: {e}")
#         state["errors"] = errs
#         raise
#     return state
from requests.exceptions import Timeout, RequestException
import random
def node_call_llm(state: ResumeState) -> ResumeState:
    max_retries = 3
    backoff_factor = 1  # seconds
    timeout = 10  # seconds

    for attempt in range(1, max_retries + 1):
        try:
            # Attempt to call the LLM
            r = call_llm_structured(state["llm"], state["jd"], state["resume_text"])
            print(r.content)
            state["raw_llm_output"] = r.content

            # Process the response
            cleaned = clean_text_v2(r.content)
            state["parsed_json"] = json.loads(cleaned)
            return state

        except Timeout as e:
            # Handle timeout errors
            error_message = f"Attempt {attempt} failed due to timeout: {e}"
            print(error_message)
            if attempt == max_retries:
                state["errors"] = state.get("errors", []) + [error_message]
                raise

        except RequestException as e:
            # Handle other request-related errors
            error_message = f"Attempt {attempt} failed due to request error: {e}"
            print(error_message)
            if attempt == max_retries:
                state["errors"] = state.get("errors", []) + [error_message]
                raise

        except Exception as e:
            # Handle unexpected errors
            error_message = f"Attempt {attempt} failed due to unexpected error: {e}"
            print(error_message)
            if attempt == max_retries:
                state["errors"] = state.get("errors", []) + [error_message]
                raise

        # Calculate exponential backoff with jitter
        sleep_time = backoff_factor * (2 ** (attempt - 1)) + random.uniform(0, 1)
        print(f"Retrying in {sleep_time:.2f} seconds...")
        time.sleep(sleep_time)

    # If all attempts fail, raise an exception
    raise Exception("All retry attempts failed.")

def node_build_report(state: ResumeState) -> ResumeState:
    try:
        resume_path_str = state["resume_path"]
        jd = state["jd"]
        parsed = state["parsed_json"]
        candidate = safe_candidate_name_from_file(Path(resume_path_str).name)
        report = {
            "candidate_name": candidate,
            "job_title": jd.get("Job_Title") or jd.get("job_title"),
            **parsed,
        }
        state["report"] = report
    except Exception as e:
        errs = state.get("errors", [])
        errs.append(f"build_report: {e}")
        state["errors"] = errs
        raise
    return state

def node_save_report(state: ResumeState) -> ResumeState:
    try:
        reports_dir = Path("reports"); reports_dir.mkdir(exist_ok=True)
        rep = state["report"]
        out_path = reports_dir / f"{rep['candidate_name']}_report.json"
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(rep, f, indent=2, ensure_ascii=False)
    except Exception as e:
        errs = state.get("errors", [])
        errs.append(f"save_report: {e}")
        state["errors"] = errs
        raise
    return state

# ---------- Graph Builder ----------
def build_resume_graph(structured_llm) -> Any:
    """
    Build a LangGraph pipeline for SINGLE resume processing:
      load_resume -> call_llm -> build_report -> save_report -> END
    The compiled graph reuses your existing helpers & prompt.
    """
    graph = StateGraph(ResumeState)
    graph.add_node("load_resume", node_load_resume)
    graph.add_node("call_llm", node_call_llm)
    graph.add_node("build_report", node_build_report)
    graph.add_node("save_report", node_save_report)

    graph.set_entry_point("load_resume")
    graph.add_edge("load_resume", "call_llm")
    graph.add_edge("call_llm", "build_report")
    graph.add_edge("build_report", "save_report")
    graph.add_edge("save_report", END)

    # In-memory checkpointing is handy for debugging / retries
    
    return graph.compile()

# ---------- Single-run Helper ----------
def run_single_resume_with_graph(jd: Dict[str, Any], resume_path: Path, structured_llm) -> Optional[Dict[str, Any]]:
    """
    Invoke the compiled LangGraph workflow for a single resume.
    Returns the final report dict or None on failure.
    """
    g = build_resume_graph(structured_llm)
    init: ResumeState = {
        "jd": jd,
        "resume_path": str(resume_path),
        "llm": structured_llm,
        "errors": [],
    }
    try:
        final_state = g.invoke(init)
        return final_state.get("report")
    except Exception as e:
        logging.error(f"Graph failed for {Path(resume_path).name}: {e}")
        return None

# ---------- UPDATED process_all ----------
def process_all(job_description_file: str, resumes_folder: str, workers: int = 2, model: str = "gpt-4o-mini", topk: Optional[int] = None, topk_frac: float = 0.4) -> None:
    reports_dir = Path("reports"); reports_dir.mkdir(exist_ok=True)

    jd = load_yaml_job_description(job_description_file)

    # Gather candidate files
    resume_files = [Path(resumes_folder) / fn for fn in os.listdir(resumes_folder)
                    if fn.lower().endswith((".pdf", ".txt"))]

    # Pre-load resume texts once for embedding prefilter
    resume_texts = [load_resume_markdown(str(p)) for p in resume_files]

    # Pre-filter via embeddings
    ranked = prefilter_resumes(jd, resume_files, resume_texts, topk=topk, topk_frac=topk_frac)
    selected_files = [p for p, _ in ranked]
    logging.info(f"Pre-filter selected {len(selected_files)}/{len(resume_files)} resumes via embeddings")

    reports: List[Dict[str, Any]] = []

    def _worker(resume_path: Path) -> Optional[Dict[str, Any]]:
        # Safer to create an LLM client per thread
        structured_llm = make_llm(model=model)
        return run_single_resume_with_graph(jd, resume_path, structured_llm)

    # Fan out across a thread pool, each calling the LangGraph workflow
    with ThreadPoolExecutor(max_workers=max(1, int(workers))) as ex:
        futs = {ex.submit(_worker, p): p for p in selected_files}
        for fut in as_completed(futs):
            p = futs[fut]
            try:
                rep = fut.result()
                if rep:
                    reports.append(rep)
                    # (Already saved inside the graph's save node)
            except Exception as e:
                logging.error(f"Failed {p.name}: {e}")

    logging.info(f"Wrote {len(reports)} reports to {reports_dir.resolve()}")


In [ ]:
job_description_file = r"C:\Users\Lenovo\resume_matcher\jd.yaml"
resumes_folder = r"C:\Users\Lenovo\resume_matcher\resumes"
topk = 5
Path("reports").mkdir(exist_ok=True)

t0 = time.time()
reports = process_all(job_description_file=job_description_file, resumes_folder=resumes_folder, topk=topk)
logging.info(f"Done processing resumes in {time.time() - t0:.1f}s")

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_25780\1506756277.py:269: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  _embedder = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
2025-09-20 21:09:34,217 INFO PyTorch version 2.7.1 available.
2025-09-20 21:09:35,239 INFO Use pytorch device_name: cpu
2025-09-20 21:09:35,241 INFO Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-09-20 21:09:40,770 INFO Pre-filter selected 5/5 resumes via embeddings
2025-09-20 21:09:44,802 INFO HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-20 21:09:44,858 INFO HTTP Request: POST https://openrouter.ai/api/v1/chat

```
Attempt 1 failed due to unexpected error: Expecting value: line 1 column 1 (char 0)
Retrying in 1.63 seconds...


2025-09-20 21:09:47,162 INFO HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-09-20 21:09:47,166 INFO Retrying request to /chat/completions in 0.472110 seconds
2025-09-20 21:09:48,793 INFO HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-09-20 21:09:48,794 INFO Retrying request to /chat/completions in 0.894162 seconds
2025-09-20 21:09:50,534 INFO HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-09-20 21:09:50,551 INFO Backing off call_llm_structured(...) for 0.6s (openai.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1758412800000'}, 'provider_name': None}}, 'user_id': 'user_32jjQAk4Yf8Ph3ZmokpzY4iHmXE

```json
{
  "matched_required_skills": ["python", "rest apis", "sql", "version control (git)", "debugging and code review"],
  "missing_required_skills": [],
  "matched_optional_skills": ["docker", "aws lambda", "flask or fastapi", "ci/cd pipelines"],
  "education_match": "true; B.Tech in Information Technology",
  "experience_match": "true; 3.5 years of relevant backend development experience",
  "keywords_matched": ["python", "rest apis", "sql", "version control (git)", "debugging and code review", "docker", "matched_optional_skills": ["docker", "aws lambda", "flask or fastapi", "ci/cd pipelines"],
  "education_match": "true; B.Tech in Information Technology",
  "experience_match": "true; 3.5 years of relevant backend development experience",
  "keywords_matched": ["python", "rest apis", "sql", "version control (git)", "debugging and code review", "docker", "aws lambda", "flask", "fastapi", "ci/cd pipelines"],
  "soft_skills_match": ["security", "monitoring"],
  "resume_summary": "Fa

2025-09-20 21:10:01,698 INFO HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-09-20 21:10:01,702 INFO Retrying request to /chat/completions in 0.476645 seconds
2025-09-20 21:10:02,339 INFO HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-09-20 21:10:02,343 INFO Retrying request to /chat/completions in 0.814946 seconds
2025-09-20 21:10:02,937 INFO HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-09-20 21:10:02,937 INFO Retrying request to /chat/completions in 0.385174 seconds
2025-09-20 21:10:04,343 INFO HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-09-20 21:10:04,349 INFO Backing off call_llm_structured(...) for 1.7s (openai.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests